In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.insert(0, os.path.abspath(".."))
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from typing import Optional
from pydantic import BaseModel
from datetime import datetime

from db import PostgresDB


In [3]:
load_dotenv()


True

In [4]:
db = PostgresDB(
    db_name=os.getenv("DB_NAME"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
)


Connected to database


In [5]:
ICU_SCHEMA = "mimiciv_icu"
HOSP_SCHEMA = "mimiciv_hosp"
ED_SCHEMA = "mimiciv_ed"


In [6]:
def create_sepsis_cohort(db: PostgresDB):
    probands = []
    query = """
        SELECT sep.subject_id, sep.stay_id, sta.hadm_id
        FROM mimiciv_derived.sepsis3 sep, mimiciv_icu.icustays sta
        WHERE sep.stay_id = sta.stay_id limit 100;
    """
    db_result = db.execute_query(query)
    for subject_id, stay_id, hadm_id in db_result:
        probands.append(
            Proband(subject_id=subject_id, stay_id=stay_id, hadm_id=hadm_id)
        )
    cohort = Cohort(participants=probands)
    return cohort


In [51]:
from cohorts import Cohort
from labevents import LabEvent, LabEventComparator, LabEventDistributions

query = """
    SELECT sep.subject_id, sta.hadm_id
    FROM mimiciv_derived.sepsis3 sep, mimiciv_icu.icustays sta
    WHERE sep.stay_id = sta.stay_id limit 100;
"""

query_b = """
    SELECT sep.subject_id, sta.hadm_id
    FROM mimiciv_derived.sepsis3 sep, mimiciv_icu.icustays sta
    WHERE sep.stay_id = sta.stay_id limit 100;
"""

cohort = Cohort.from_query(query, db)


In [52]:
cohort.initialize_data()


In [9]:
cohort.labevents[0]

LabEvent(labevent_id=279722, item_id=50812, subject_id=10023771, hadm_id=20044587, specimen_id=23248642, charttime=datetime.datetime(2113, 8, 25, 11, 34), value='INTUBATED.', valuenum=None, valueuom=None, ref_range_lower=None, ref_range_upper=None, flag=None, priority=None, comments='___', label='Intubated', fluid='Blood', category='Blood Gas')

In [53]:
lab_distr = LabEventDistributions(cohort.labevents)

In [54]:
lab_comp = LabEventComparator(lab_distr)

In [41]:
a = "1.5".lstrip("-").isdecimal()
a

False

In [76]:
lab_comp.compare(
    cohort.similarity_encounters[1].labevents,
    cohort.similarity_encounters[0].labevents,
    scale_by_distribution=True,
)


0.20390399635864842

In [101]:
lab_comp.compare(
    cohort.similarity_encounters[5].labevents,
    cohort.similarity_encounters[5].labevents,
)


50868 12 10.625 0.6881428006618527
50868 12 10.625 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 10.625 12 0.6881428006618527
50868 12 10.625 0.6881428006618527
50868 12 10.625 0.6881428006618527
50868 12 10.625 0.6881428006618527
50868 12 10.625 0.6881428006618527
[1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 0.6881428006618527, 1.0, 0.6881428006618527, 0.6881428006618527, 0.6881428006618527, 0.6881428006618527, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6881428006618527, 1.0, 1.0, 1.0, 1.0]

0.9532214200992779

In [75]:
for a in cohort.similarity_encounters[0].labevents:
    for b in cohort.similarity_encounters[1].labevents:
            one = lab_comp.compare([a], [b], scale_by_distribution=True)
            rwo = lab_comp.compare([b], [a], scale_by_distribution=True)
            if not one == rwo:
                print("WEEOE")
                print(one, rwo)

Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249
Multiple units found for item_id: 51249


In [13]:
from cohorts import EncounterComparator

comp = EncounterComparator()


In [14]:
cohort.similarity_encounters[0].demographics


Demographics(subject_id=10001884, age=68, gender='F')